# Converting Rosanne's magnification files to magnified light curves and with the PLASTICC/SNANA format

### USER

In [1]:
# Count the number of times I run this cell. The number
# is used to label the output files.
count_run = 1

# -------------------------------

## Kind of lensing events producing the magnification:
# KindLensEvent = 'PointSource'
KindLensEvent = 'Binary'

# -------------------------------

# Dir of simulated LSST star catalog
DirStarCatalog = "/Users/arturo/Documents/Research/LSST/Catalog/Subsamples/\
Region_ra_0.0_360.0_dec_-90.0_40.0/"

# Dir of magnification files (those provided by Rosanne Di Stefano)
if KindLensEvent == 'PointSource':
    DirLensFiles = "/Users/arturo/Documents/Research/LSST/Rosanne/2018_07_25_Point/"
elif KindLensEvent == 'Binary':
    DirLensFiles = "/Users/arturo/Documents/Research/LSST/Rosanne/2018_07_24_Binary/"

# Dir of expected lensing events in LSST catalog.
DirExpectedEvents = "/Users/arturo/Dropbox/Research/Articulos/14_Lensing/OGLE/\
Count_events_stars_size_10.0/Region_ra_0.0_360.0_dec_-90.0_40.0/"

# -------------------------------
#       Dir save output
# Documents
# DirSaveOutput = "/Users/arturo/Documents/Research/LSST/Rosanne/\
# 2018_07_16/2_output/"

# Dropbox
# DirSaveOutput = "/Users/arturo/Dropbox/Research/Articulos/14_PLASTICC/\
# MockData/Samples/2018_07_16/2_output/"

# Dropbox: Production
DirSaveOutput = "/Users/arturo/Dropbox/Research/Articulos/\
14_Lensing/MockData/Production_%s_2/"%KindLensEvent

# -------------------------------
# Given that javascript doesn't work in JupyterLab, I have to set the 
# name of the notebook by hand
NotebookName = 'Create_magnified_LCs.ipynb'



### Automatic

In [2]:
import numpy as np
import pandas as pd
import os # To use command line like instructions
import glob # To read the files in my directory
from matplotlib import pyplot as plt

5+6

11

#### Get the name of this ipython notebook
To print it in the output text files as reference.

In [3]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "NotebookName = " + "'"+thename+".ipynb"+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [4]:
print(NotebookName)

# Given that javascript doesn't work in JupyterLab, I have to set the 
# name of the notebook by hand

Create_magnified_LCs.ipynb


In [5]:
# Get the current date and time
import datetime 

# Read the time and date now
now = datetime.datetime.now()

### Magnification & blending

In [6]:
# Function to convert from magnification to apparent magnitude

def Magnification2mag(Magf, mo):
        
    mag_int = mo - 2.5*np.log10(Magf) 
    return mag_int

print '# Test:'
print "#", Magnification2mag(2.14118954, 19.46)

# Test
# 18.6333622172
# 18.633362217176515

# Test:
# 18.633362217176515


In [7]:
# Blending functions

def kappa_fun(mag_s, mag_2):
    'The kappa function'
    power_val = (mag_s - mag_2)/2.5
    return np.power(10.0, power_val)

#------------------------------------

def ff_fun(kappa):
    return 1.0/(kappa + 1.0)

#------------------------------------
# Observed total magnification from the 2 stars combined

def AA_total_blend_fun(AA, ff):
    return AA*ff + (1.0-ff)

#------------------------------------
# apparent magnitude from combined stars. No magnified

def appmag_base_fun(mag_s, kappa):
    return mag_s - 2.5*np.log10(1.0+kappa)

#------------------------------------
# apparent magnitude from combined stars. Magnified

def appmag_magnify_fun(mag_s, AA, kappa):
    return mag_s - 2.5*np.log10(AA+kappa)

## Read files

In [8]:
# Read the LSST star catalogue file with the magnitudes in ugrizY bands

starsdata_file = 'LSST_ra_0.0_360.0_dec_-90.0_40.0_Jump_100_.dat'

starsdata = np.genfromtxt(DirStarCatalog+starsdata_file,
                dtype=[float,float,float,float,float,
                       float,float,float]) 

print "# File: '%s'"%starsdata_file
print "# %s stars data found in the file"%len(starsdata)

# File: 'LSST_ra_0.0_360.0_dec_-90.0_40.0_Jump_100_.dat'
# 169229 stars data found in the file

# File: 'LSST_ra_0.0_360.0_dec_-90.0_40.0_Jump_100_sample.dat'
# 4999 stars data found in the file

# File: 'LSST_ra_0.0_360.0_dec_-90.0_40.0_Jump_100_.dat'
# 169229 stars data found in the file


In [9]:
# Sample of the file
starsdata[:5]

array([(249.2108339, -53.7600708, 16.21739, 14.93847, 14.53348, 14.35644, 14.19864, 14.09501),
       (262.8786919, -16.0070474, 18.41886, 17.15392, 16.62919, 16.38289, 16.20383, 16.10269),
       (267.8598182, -28.6188357, 27.47009, 24.50058, 22.16687, 20.72908, 19.54815, 18.84788),
       (254.9826929, -35.7618161, 19.72312, 17.90901, 16.87163, 16.30968, 15.86593, 15.61027),
       (242.4658103, -50.4776049, 30.9791 , 30.8132 , 27.20913, 24.40215, 22.17582, 20.8344 )],
      dtype=[('f0', '<f8'), ('f1', '<f8'), ('f2', '<f8'), ('f3', '<f8'), ('f4', '<f8'), ('f5', '<f8'), ('f6', '<f8'), ('f7', '<f8')])

In [10]:
"""
array([ ( 269.9092004, -28.3269497,  20.99688,  19.00108,  17.79934,  17.13335,  16.60385,  16.29794),
       ( 267.0523731, -32.4916441,  28.21959,  25.46772,  22.80995,  21.13215,  19.76051,  18.95455),
       ( 273.7255786, -16.1384652,  29.85822,  28.62047,  25.17983,  22.84342,  21.00829,  19.91717),
       ( 276.9094877, -18.2496449,  22.17216,  20.27804,  19.11986,  18.48972,  18.00777,  17.73689),
       ( 273.004142 , -16.2747554,  29.37052,  27.52682,  24.3281 ,  22.22627,  20.55627,  19.57183)], 
      dtype=[('f0', '<f8'), ('f1', '<f8'), ('f2', '<f8'), ('f3', '<f8'), ('f4', '<f8'), ('f5', '<f8'), ('f6', '<f8'), ('f7', '<f8')])
"""
0

0

#### Rosanne

In [11]:
# Define the name of Rosanne's MAGNIFICATION file (the huge file)

if KindLensEvent == 'PointSource':
    MagnificationData = 'tau.dist_lcs_new'
    
elif KindLensEvent == 'Binary':
    MagnificationData = 'bin_lc_results_10_50_n.dms'

In [12]:
# Read the lensing METADATA file:

if KindLensEvent == 'PointSource':
    lens_metadata_file = 'tau.dist_information_new'
    lens_metadata = np.genfromtxt(DirLensFiles+lens_metadata_file #)
                             ,dtype=[int,int,float,float,float,float])
elif KindLensEvent == 'Binary':
    lens_metadata_file = 'bin_lc_information_10_50_n.dms'
    lens_metadata = np.genfromtxt(DirLensFiles+lens_metadata_file)

print "# File: '%s'"%lens_metadata_file
print "# %s LCs data found in the file. "%len(lens_metadata)

# tau_dist_information.txt
# 150000 LCs data found in the file.

# tau_dist_information_sample.txt
# 10000 LCs data found in the file. 

# File: 'tau_dist_information_sample.txt'
# 40000 LCs data found in the file. 

# File: 'bin_lc_information_10_50_n.dms'
# 30682 LCs data found in the file. 

# File: 'bin_lc_information_10_50_n.dms'
# 30682 LCs data found in the file. 


In [13]:
# Sample of the array
lens_metadata[:5]

array([[ 1.00000e+00,  2.56000e+02,  1.13900e+00,  1.23700e+01,
         3.34700e+04,  3.57700e+10,  8.00000e+03,  2.61400e+03,
         9.78300e-05,  4.56512e-03,  1.07608e+03,  1.42969e+00,
         1.55605e+00,  6.89732e-02,  3.43659e-03,  1.12853e-03,
        -1.00000e+00,  5.30000e-02,  9.95000e-01],
       [ 2.00000e+00,  2.90000e+02,  1.75890e+00,  2.19400e+01,
         5.57800e+04,  1.05800e+11,  8.00000e+03,  7.83500e+03,
         9.65000e-05,  4.34635e-01,  9.89663e+01,  4.49896e-01,
         1.89241e+00,  1.33042e+00,  3.14475e-01,  1.20160e-01,
        -1.00000e+00, -1.56000e-01, -7.10000e-01],
       [ 3.00000e+00,  2.28000e+02,  1.12210e+00,  9.45000e+00,
         2.58500e+04,  2.11100e+10,  8.00000e+03,  6.09400e+03,
         2.47600e-05,  1.92638e-03,  3.22291e+02,  8.13504e-01,
         1.50009e+00,  1.75916e-01,  1.51556e-03,  4.10817e-04,
         1.00000e+00,  1.38600e+00,  3.93000e-01],
       [ 4.00000e+00,  7.67000e+02,  1.69943e+01,  8.28000e+00,
         1.8210

In [14]:
"""
array([(1, 197,   48.48 ,  1.602  , -0.1665  ,   1.112),
       (2,  30,    3.692,  3.059  ,  0.1651  ,   1.015),
       (3,  84,  134.1  ,  2.219  ,  0.1174  ,   1.044),
       (4,  84,   40.01 ,  2.202  , -0.005246,   1.045),
       (5, 480,    3.228,  0.04616, -0.05398 ,  21.596)], 
      dtype=[('f0', '<i8'), ('f1', '<i8'), ('f2', '<f8'), ('f3', '<f8'), ('f4', '<f8'), ('f5', '<f8')])
"""
0

0

In [15]:
# Read the table containing the number of expected events per square degree
# inferred from OGLE sky map.

expectedEventsLSST_file = "Table_LSST_stars_events_interpol_left_.dat"
expectedEventsLSST_np = np.genfromtxt(DirExpectedEvents+expectedEventsLSST_file)

print "# File: '%s'"%expectedEventsLSST_file
print "# %s boxes in this (ra,dec) grid table."%len(expectedEventsLSST_np)

TotalNumEvents = sum(expectedEventsLSST_np[:,5])
print "# %i: Total number of expected lensing events in the whole LSST sky."%TotalNumEvents

# Find the maximum and minimum number of events found in a box
# This will be used to compute and write the density of events weigth.
MaxNumLensEvents = max(expectedEventsLSST_np[:,5])
MinNumLensEvents = min(expectedEventsLSST_np[:,5])

print "# %i: Max number of events in a box."%MaxNumLensEvents
print "# %i: Min number of events in a box."%MinNumLensEvents

# File: 'Table_LSST_stars_events_interpol_left_.dat'
# 468 boxes in this (ra,dec) grid table.
# 179839: Total number of expected lensing events in the whole LSST sky.
# 19980: Max number of events in a box.
# 4: Min number of events in a box.

# File: 'Table_LSST_stars_events_interpol_left_.dat'
# 468 boxes in this (ra,dec) grid table.
# 179839: Total number of expected lensing events in the whole LSST sky.
# 19980: Max number of events in a box.
# 4: Min number of events in a box.


In [16]:
expectedEventsLSST_np[:5]

array([[  0.,  10., -90., -80., 600.,   6.],
       [ 10.,  20., -90., -80., 700.,   7.],
       [ 20.,  30., -90., -80., 900.,   9.],
       [ 30.,  40., -90., -80., 600.,   6.],
       [ 40.,  50., -90., -80., 400.,   4.]])

In [17]:
"""
array([[   250.,    251.,    -40.,    -39.,   6700.,     69.],
       [   251.,    252.,    -40.,    -39.,  11100.,    114.],
       [   252.,    253.,    -40.,    -39.,  12200.,    131.],
       [   253.,    254.,    -40.,    -39.,  12000.,    123.],
       [   254.,    255.,    -40.,    -39.,   7700.,     79.]]) """
0

0

In [18]:
# Create a copy array of "expectedEventsLSST_np"
# This will be used to save the counting in each square

count_expectedEvents_np = np.copy(expectedEventsLSST_np)

print "%s boxes in this file."%len(count_expectedEvents_np)
count_expectedEvents_np[:5]

468 boxes in this file.


array([[  0.,  10., -90., -80., 600.,   6.],
       [ 10.,  20., -90., -80., 700.,   7.],
       [ 20.,  30., -90., -80., 900.,   9.],
       [ 30.,  40., -90., -80., 600.,   6.],
       [ 40.,  50., -90., -80., 400.,   4.]])

In [19]:
# Reset to zero all the numbers of events in the last column

for i in range(len(count_expectedEvents_np)):               
    count_expectedEvents_np[i][5] = 0.0
    
#-------------------
count_expectedEvents_np[:5]

array([[  0.,  10., -90., -80., 600.,   0.],
       [ 10.,  20., -90., -80., 700.,   0.],
       [ 20.,  30., -90., -80., 900.,   0.],
       [ 30.,  40., -90., -80., 600.,   0.],
       [ 40.,  50., -90., -80., 400.,   0.]])

In [20]:
# count_run = 1 

### MAIN LOOP

In [35]:
# Create folder to save the plots

#- Force the creation of the directory to save the plots.
#- "If the subdirectory does not exist then create it"
import os # To use command line like instructions

if not os.path.exists(DirSaveOutput+"%s_plots"%count_run): 
    os.makedirs(DirSaveOutput+"%s_plots"%count_run)

In [36]:
# Initialize the timing of computation
now_start = datetime.datetime.now()
time_start = now_start.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Starting time of computing: %s"%time_start

# Starting time of computing: 2018-08-01 (yyyy-mm-dd); 23:36 hrs.


In [37]:
# Write to a text file all the details while 
# running this script.
debug = False

# Print on the screen below this cell all the verbose
# when I'm in debugging mode? 
# When 'debug = False' then this variable is not read
# so I does't matter its value.
VerboseOnScreen = False

#--------------------------------------------------

# Plot the output light curves?
plot_LightCurves = True

# Overplot the LCs assuming no blending?
plot_WithNoBlending = True

# Plot every n-th SUCESSFUL LCs
PlotEvery_n = 1

#--------------------------------------------------
#    Some parameters and settings

# Difference in u magnitude as a criterium to write down
# that row in the text file. This is helpful to avoid writting a lot of
# rows with exactly the same values of the magnitudes in ugrizY bands.
diffmag_fix = 0.001

# Rounding the magnified app mag
if diffmag_fix == 0.01:
    round_appmag = 2
elif diffmag_fix == 0.001:
    round_appmag = 3

# Maximum percentage of time of a LC with magnitude larger
# than 16 mags
Porcentage_maxTime = 10.0 # percentage

# Maximum apparent magnitude. LSST saturates for
# objects brighter than 16 mag.
maxMag = 16.0 # mag

# Limit of dimmest objects observed by LSST
max_dim_mag = 24.0  # mag

# Maxiumum percentaje of observations dimmer than the dimmest limit
# to be account and written in the PLASTICC file.
Percentage_obs_MaxDim = 10.0  # percentage

# Minimal baseline magnitude to be considered to run the
# rest of the loop. This help to speed up the search for events
# with magnitudes in the specified range.
min_baseline_dim_mag  = 25.2
    
## Minimal number of observations (that passed all the cuts) in
## a given light curve to be considered and written in the
# PLASTICC file.
MinNumGoodObs = 10

## Minimum -total- magnification once I blend 2 stars:
Min_TotalMagnific = 1.11

## Percentage to increase the time in the last date
percent_increaseLastDate =  15.0 # float

#------------------------

## Number of desired number of events to write down in the PLASTICC
# file.
TargetNumberEvents = 8000;
# TargetNumberEvents = 100;

## Loop size. Number of iterations in the main "for" loop below.
# LoopSize = 1000000;
LoopSize = 100;

## Define (automatically) the percentage to decrease the total number
# of expected events per box based on the "TargetNumberEvents" and the 
# total number of expected events in the full LSST sky.
events_percent_reduction = ((TargetNumberEvents*100.)/TotalNumEvents) - 0.2

## Reset the index to read each row in the LSST star file.
# The "-4" helps to define the 2nd star for blending.
ResetStarFile = len(starsdata)-4
# ResetStarFile = 4000
# ResetStarFile = 10

## Reset the index used to read each row in Rosanne -metadata- file
# when it gets the following value (= line in Rosanne -metadata- file)
# ResetLensFile = len(lens_metadata)-100
# ResetLensFile = 10
if KindLensEvent == 'PointSource':ResetLensFile = 1700
elif KindLensEvent == 'Binary': ResetLensFile = 1922

# Seed to generate random numbers to pick the LSST star
        
# Initial value of the index that read the LSST star catalog.
# Create an offset in the reading of the LSST catalog to
# ensure don't generate exactly previous already
# generated events, and also to use stars from other parts
# of the LSST sky.    
# initial_index_starsFile = 100000
initial_index_starsFile = 0

## Hack to resume from a previous run.
# Resume the counter to the last sucessful stars in a
# from a previous run:
# count_succesful_stars = 1462; 
count_succesful_stars = 0 


#####################################################
#####################################################

#------------------------------------------
# Reset to zero all the numbers of events in the last column

for i in range(len(count_expectedEvents_np)):
    count_expectedEvents_np[i][5] = 0.0

#------------------------------------------

textfile_1 = open(DirSaveOutput+
                  '%s_Lensing_lc_PLASTICC_format_.dat'%count_run, 'w')
textfile_2 = open(DirSaveOutput+
                  '%s_LSST_lensingEvents_RA_DEC_.dat'%count_run, 'w')
textfile_3 = open(DirSaveOutput+
                  '%s_Report_.log'%count_run, 'w')
# if debug == True:
#     textfile_4 = open(DirSaveOutput+
#                   '%s_Report_debug_.log'%count_run, 'w')
    

#  TEXT
now = datetime.datetime.now() # Read the time and date right now
text_timenow = now.strftime("%m/%d/%Y")
text_Date   = 'COMMENT: Created by Rosanne Di Stefano & \
Arturo Avelino on %s\n'%text_timenow

text_script = '# Script used: %s \n'%NotebookName
text_line = '#'+'-'*30 + '\n'

text_000 = "# SURVEY: LSST\n"
text_002 = "# FILTERS: ugrizY\n"

if KindLensEvent == 'PointSource':
    text_comment_2 = "COMMENT: Magnified light curves from point lens\n"
    text_004 = "# MODEL: point-lens/point source\n"
    text_006 = "# MODEL_PARNAMES: tau_e,u_0\n"
elif KindLensEvent == 'Binary':
    text_comment_2 = "COMMENT: Magnified light curves from binary lens\n"
    text_004 = "# MODEL: binary-lens/point-source\n"
    text_006 = "# MODEL_PARNAMES: tau,v,re_in,d_s,d_l,teta_e,mass,porb,alpha,\
beta,ratio,m1,m2,clock,i_p,omega\n"
    
text_00710 = "# RECUR_TYPE: NON-RECUR\n"
text_00720 = "# NEVENT:  \n"

text_008 = "# RA       DEC \n"

text_00605 = "# LSST star catalogue file: '%s'. \n"%starsdata_file
text_00610 = "# %s \n"%DirStarCatalog
text_00615 = "# Lensing metadata file from Rosanne: '%s' .\n"%lens_metadata_file
text_00620 = "# %s .\n"%DirLensFiles
text_00622 = "# Lensing main datafile from Rosanne: '%s' .\n"%MagnificationData
text_00624 = "# %s .\n"%DirLensFiles
text_00625 = "# Number of expected events per square degree file: '%s' .\n"%expectedEventsLSST_file
text_00630 = "# %s .\n"%DirExpectedEvents


text_008010 = "# %s = Kind of lensing events producing the magnification.\n"%KindLensEvent
text_008014 = "# %s = Minimum total magnification.\n"%Min_TotalMagnific
text_00802 = "# %s = Loop size. This is the number of iterations \
to run in the main loop. \n"%(LoopSize)
text_00804 = "# %s = Target number of events to write \
down in the PLASTICC file.\n"%(TargetNumberEvents)
text_0080405 = "# %i = Initial value of the index that read the LSST star catalog. \
Useful to resume from a previous run.\n"%(initial_index_starsFile)
text_0080410 = "# %.5f = Percentage to decrease the total number of expected \
events per box based on the 'TargetNumberEvents' and the total \
number of expected events in the full LSST sky.\n"%(events_percent_reduction)
text_0080420 = "# %i = Total number of boxes to fill.\n"%(len(expectedEventsLSST_np))
text_0080430 = "# %i = Original total number of events in all the boxes.\n"%(TotalNumEvents)
#------ 
text_00806 = "# %s = Reset the index to read each row in Rosanne LCs file \
when it reaches this value.\n"%(ResetLensFile)
text_00808 = "# %s = Reset the index to read each row in the LSST star file \
when it reaches this value.\n"%(ResetStarFile)
text_00810 = "# %s = Minimal number of observations (that passed all the cuts) \
in a given light curve to be considered and written in the PLASTICC file.\n"%(MinNumGoodObs)
text_00812 = "# %s = Maximum apparent magnitude. LSST saturates for \
objects brighter than 16 mag.\n"%(maxMag)
text_00814 = "# %s = Maximum percentage of time of a LC with magnitude \
larger than %s mags.\n"%(Porcentage_maxTime, maxMag)
text_00816 = "# %s = Limit of dimmest objects observed by LSST.\n"%(max_dim_mag)
text_00818 = "# %s = Maxiumum percentaje of observations dimmer than the \
dimmest limit to be account and written in the PLASTICC file..\n"%(Percentage_obs_MaxDim)
text_00819 = "# %s = Minimal baseline magnitude to be considered to run the \
rest of the loop. This help to speed up the search for events with magnitudes \
in the specified range. \n"%min_baseline_dim_mag
text_00820 = "# %s = Difference in magnitude in u-band as a criterium to \
write down that row in the text file. \n"%(diffmag_fix)
text_00822 = "# %s = Percentage to increase the time in the last date. \n"%percent_increaseLastDate


text_00824 = "#   Light curves written in the PLASTICC file \n"
text_00828 = "#     j     PLASTICCevent  LSSTstar RosanneLCNumber index_lensFile \
#Event  #ExpEvent TotalEvent BoxNum  RA        DEC       InitialRow  1stNum  NumberRows  \
Star_A_Peak        r-A_Peak  Band \n"

#============================================================
# WRITTING in the files

#    PLASTICC file
textfile_1.write(text_000[2:]);textfile_1.write(text_002[2:]);
textfile_1.write(text_004[2:]);textfile_1.write(text_00710[2:]);
textfile_1.write(text_00720[2:]);
textfile_1.write(text_006[2:]);
textfile_1.write(text_Date); textfile_1.write(text_comment_2); 

#----------------

textfile_2.write(text_000);textfile_2.write(text_002);textfile_2.write(text_004);
textfile_2.write(text_006);textfile_2.write("# "+text_Date);
textfile_2.write(text_script);textfile_2.write(text_line);

textfile_2.write(text_00605);textfile_2.write(text_00610);textfile_2.write(text_00615);
textfile_2.write(text_00620);textfile_2.write(text_00625);textfile_2.write(text_00630);
textfile_2.write(text_line);
textfile_2.write(text_008010);textfile_2.write(text_008014);
textfile_2.write(text_00802);textfile_2.write(text_00804);
textfile_2.write(text_0080410);
textfile_2.write(text_0080420);textfile_2.write(text_0080430);
textfile_2.write(text_00806);textfile_2.write(text_00808);
textfile_2.write(text_00810);textfile_2.write(text_00812);
textfile_2.write(text_00814);textfile_2.write(text_00816);
textfile_2.write(text_00818);textfile_2.write(text_00819);
textfile_2.write(text_00820);textfile_2.write(text_00822);
textfile_2.write(text_line);

textfile_2.write(text_008);

#------------
#       REPORT file

textfile_3.write(text_000);textfile_3.write(text_002);textfile_3.write(text_004);
textfile_3.write(text_006);textfile_3.write("# "+text_Date);
textfile_3.write(text_script);
textfile_3.write(text_line);

textfile_3.write(text_00605);textfile_3.write(text_00610);textfile_3.write(text_00615);
textfile_3.write(text_00620);textfile_3.write(text_00622);textfile_3.write(text_00624);
textfile_3.write(text_00625);textfile_3.write(text_00630);
textfile_3.write(text_line);

textfile_3.write(text_008010);textfile_3.write(text_008014);
textfile_3.write(text_00802);textfile_3.write(text_00804);
textfile_3.write(text_0080405);
textfile_3.write(text_0080410);
textfile_3.write(text_0080420);textfile_3.write(text_0080430);
textfile_3.write(text_00806);textfile_3.write(text_00808);
textfile_3.write(text_00810);textfile_3.write(text_00812);
textfile_3.write(text_00814);textfile_3.write(text_00816);
textfile_3.write(text_00818);textfile_3.write(text_00819);
textfile_3.write(text_00820);textfile_3.write(text_00822);
textfile_3.write(text_line);
textfile_3.write(text_00824);textfile_3.write(text_00828);

#------------
#       REPORT with debug mode


#------------

print "# Number of useful rows written in the text \
file for a given star,"
print "# it is, when there is a change in magnitude \
at least > %s mag."%diffmag_fix

#-------------------------------------------------
#    Reset variables.

# Create an offset in the reading of the LSST catalog to
# ensure don't generate exactly previous already 
# generated events, and also to use stars from other parts
# of the LSST sky.
index_starsFile = initial_index_starsFile

index_lensFile = 0; initial_row = 0;
count_boxesFull = 0;

#-------------------------------------------------
# Loop over the stars in the LSST catalog.

for jj in range(LoopSize):
    
    # Reset the variable used to read each row in the LSST
    # catalog once it reach certain limit.
    if index_starsFile == ResetStarFile:
        index_starsFile = 0
        
    # Reset the variable used to read each row in the Rosanne
    # light-curve file once it reach certain limit.
    if index_lensFile == ResetLensFile:
        index_lensFile = 0; initial_row = 0;
        
    # Reset the flag that indicates that 'initial_row' 
    # was updated.
    initial_row_updateFlag = 0
    
    #---------------------------------
    
    # Apparent magnitude star in different bands.
    # Source star
    magStar_u1 = starsdata['f2'][index_starsFile]
    magStar_g1 = starsdata['f3'][index_starsFile]
    magStar_r1 = starsdata['f4'][index_starsFile]
    magStar_i1 = starsdata['f5'][index_starsFile]
    magStar_z1 = starsdata['f6'][index_starsFile]
    magStar_y1 = starsdata['f7'][index_starsFile]
    
    flag_baseline_dim_mag = 0 # reset
    # Check if any band is brighter than the minimum baseline app mag
    if   magStar_u1 < min_baseline_dim_mag: flag_baseline_dim_mag = 1
    elif magStar_g1 < min_baseline_dim_mag: flag_baseline_dim_mag = 1
    elif magStar_r1 < min_baseline_dim_mag: flag_baseline_dim_mag = 1
    elif magStar_i1 < min_baseline_dim_mag: flag_baseline_dim_mag = 1
    elif magStar_z1 < min_baseline_dim_mag: flag_baseline_dim_mag = 1
    elif magStar_y1 < min_baseline_dim_mag: flag_baseline_dim_mag = 1
        
    if debug: 
        text_008305 = "# %s LSST star.\n"%(index_starsFile+1)
        text_008307 = "# %s Rosanne LC.\n"%(index_lensFile+1)
        text_008310 = "# At least 1 band is brigther than the limit %s mag \
(No cuts applied yet). \n"%(
            min_baseline_dim_mag)
        textfile_3.write(text_line);
        textfile_3.write(text_008305);textfile_3.write(text_008307);
        textfile_3.write(text_008310);
        
    #---------------------------------
    
    # Peak magnification of the source star:
    if KindLensEvent == 'PointSource':
        AA_at_peak_int = lens_metadata[index_lensFile][5]
    elif KindLensEvent == 'Binary':
        AA_at_peak_int = lens_metadata[index_lensFile][2]
        
    if debug:
        text_008320 = "# %.4f is the peak magnification of the initial \
single star (No cuts applied yet). \n"%(AA_at_peak_int)
        textfile_3.write(text_008320);
        
    #------------------------------------------
    # Number of observations in that light curve:
    number_rows = int(lens_metadata[index_lensFile][1])

    if debug:
        text_02202 = "B: %7s = jj  %7s = index_starsFile  \
%7s = index_lensFile.\n"%(jj, index_starsFile, index_lensFile)
        text_02206 = "Number_rows = %s | index_lensFile = %s. \n"%(
            number_rows, index_lensFile)
        textfile_3.write(text_02202);textfile_3.write(text_02206);
        if VerboseOnScreen:
            print text_02202,text_02206
        
    ###############################################################
    
    # Consider the star only if its baseline apparent magnitude is brighter 
    # than a given dim limit, AND with a peak initial magnification of the
    # single star larger than a threshold.
    
    if (flag_baseline_dim_mag == 1 and
        AA_at_peak_int >= Min_TotalMagnific):
    
        ra  = starsdata['f0'][index_starsFile]
        dec = starsdata['f1'][index_starsFile]        

        if debug:
            text_013 = "      Star %s in LSST catalog.\n"%(index_starsFile+1) 
            text_015 = "A: %7s = jj  %7s = index_starsFile  \
%7s = index_lensFile.\n"%(jj, index_starsFile, index_lensFile) 
            text_01502 = "ra = %.2f, dec = %.2f.\n"%(ra, dec)

            textfile_3.write(text_013);
            textfile_3.write(text_015);textfile_3.write(text_01502);

            if VerboseOnScreen:
                print text_013,text_015, text_01502

        #####################################################

        # Loop over all the boxes.

        # Reset counter
        box_row_index = 0;

        # Find out in what box the current star is located,
        # then count it for that box.
        for i4 in range(len(count_expectedEvents_np)):

            ra_min_int = count_expectedEvents_np[i4][0]
            ra_max_int = count_expectedEvents_np[i4][1]
            de_min_int = count_expectedEvents_np[i4][2]
            de_max_int = count_expectedEvents_np[i4][3]
            num_events_int = count_expectedEvents_np[i4][5]
            num_events_lim_fix = expectedEventsLSST_np[i4][5]
            num_events_lim_int = int(float(expectedEventsLSST_np[i4][5])*
                                     (events_percent_reduction*0.01))
            
            # Find out in what box the current star is located,
            # then count it for that box.
            if (ra > ra_min_int and ra < ra_max_int and
                dec > de_min_int and dec < de_max_int):

                # Row in the array corresponding to the box where
                # the star is located. If this star passes the next
                # cutoffs then I'll update the counter for this box
                # in the 'expectedEventsLSST_np' array
                box_row_index = i4

                if debug:
                    text_016 = "Star found on box %s. \n"%box_row_index
                    text_019 = "ra = %.2f, dec = %.2f | %i of %i events at this box | \
(%.0f,%.0f,%.0f,%.0f) = (ra,dec) box.\n"%(ra, dec, num_events_int,
                num_events_lim_int,
                ra_min_int, ra_max_int, de_min_int, de_max_int)

                    textfile_3.write(text_016);textfile_3.write(text_019);

                    text_01902='';text_01906=''; # reset
                    if num_events_int == num_events_lim_int:
                        text_01902 = "\n    I FOUND YOU! \n"
                        text_01906 = "%i num_events = %i num_events_lim. \n\n"%(
                        num_events_int, num_events_lim_int)
                        textfile_3.write(text_01902);textfile_3.write(text_01906);

                    if VerboseOnScreen:
                        print text_016,text_019
                        print text_01902,text_01906

                # If I've already found the location of the star
                # in box, then break this loop and move on to the
                # rest of this script.
                break

        ##########################################################

        # Compute the total -maximum- magnification from the
        # 2 stars in all bands.

        # 2nd star = the next in the LSST catalog.
        magStar_u2 = starsdata['f2'][index_starsFile+1]
        magStar_g2 = starsdata['f3'][index_starsFile+1]
        magStar_r2 = starsdata['f4'][index_starsFile+1]
        magStar_i2 = starsdata['f5'][index_starsFile+1]
        magStar_z2 = starsdata['f6'][index_starsFile+1]
        magStar_y2 = starsdata['f7'][index_starsFile+1]

        # Compute kappa for the blended stars:
        kappa_u_int = kappa_fun(magStar_u1, magStar_u2)
        kappa_g_int = kappa_fun(magStar_g1, magStar_g2)
        kappa_r_int = kappa_fun(magStar_r1, magStar_r2)
        kappa_i_int = kappa_fun(magStar_i1, magStar_i2)
        kappa_z_int = kappa_fun(magStar_z1, magStar_z2)
        kappa_y_int = kappa_fun(magStar_y1, magStar_y2)

        ff_u_int = ff_fun(kappa_u_int);
        ff_g_int = ff_fun(kappa_g_int);
        ff_r_int = ff_fun(kappa_r_int);
        ff_i_int = ff_fun(kappa_i_int);
        ff_z_int = ff_fun(kappa_z_int);
        ff_y_int = ff_fun(kappa_y_int);

        # Total -maximum- magnification
        AA_total_max_u_int = AA_total_blend_fun(AA_at_peak_int, ff_u_int);
        AA_total_max_g_int = AA_total_blend_fun(AA_at_peak_int, ff_g_int);
        AA_total_max_r_int = AA_total_blend_fun(AA_at_peak_int, ff_r_int);
        AA_total_max_i_int = AA_total_blend_fun(AA_at_peak_int, ff_i_int);
        AA_total_max_z_int = AA_total_blend_fun(AA_at_peak_int, ff_z_int);
        AA_total_max_y_int = AA_total_blend_fun(AA_at_peak_int, ff_y_int);

        flag_dim_AA = "x"; #reset
        if   AA_total_max_u_int>Min_TotalMagnific:flag_dim_AA = "u"
        elif AA_total_max_g_int>Min_TotalMagnific:flag_dim_AA = "g"
        elif AA_total_max_r_int>Min_TotalMagnific:flag_dim_AA = "r"
        elif AA_total_max_i_int>Min_TotalMagnific:flag_dim_AA = "i"
        elif AA_total_max_z_int>Min_TotalMagnific:flag_dim_AA = "z"
        elif AA_total_max_y_int>Min_TotalMagnific:flag_dim_AA = "y"

        text_20004 = '' # reset
        if flag_dim_AA != "x":
            text_20004 = "Flag activated!: At least band '%s' \
has a total magnification larger than %s.\n"%(flag_dim_AA,Min_TotalMagnific)

        text_021005 = "Magnification: star only = %.5f . Total: u = %.5f , \
g = %.5f , r = %.5f , i = %.5f , z = %.5f , y = %.5f .\n"%(
        AA_at_peak_int, AA_total_max_u_int, AA_total_max_g_int,
        AA_total_max_r_int, AA_total_max_i_int, AA_total_max_z_int,
        AA_total_max_y_int)

        if debug: 
            textfile_3.write(text_20004), textfile_3.write(text_021005)
            if VerboseOnScreen: 
                print text_20004, text_021005

        #==========================================================

        # OLD ---->
        """ 
        magStar_u_s = starsdata['f2'][index_starsFile] # source star
        magStar_u_2 = starsdata['f2'][index_starsFile+1] # 2nd star

        # Compute some intermediate values:
        kappa_int = kappa_fun(magStar_u_s, magStar_u_2)
        ff_int = ff_fun(kappa_int)

        # Peak magnification of the source star:
        if KindLensEvent == 'PointSource':
            AA_at_peak_int = lens_metadata[index_lensFile][5]
        elif KindLensEvent == 'Binary':
            AA_at_peak_int = lens_metadata[index_lensFile][2]

        # total -maximum- magnification
        AA_total_max_int = AA_total_blend_fun(AA_at_peak_int, ff_int)

        if debug:
            text_02010 = "A2: %7s = jj  %7s = index_starsFile  \
 %7s = index_lensFile.\n"%(jj, index_starsFile, index_lensFile) 
            text_021 = "Magnification: source star only = %.5f , total = %.5f.\n"%(
            AA_at_peak_int, AA_total_max_int)

            textfile_3.write(text_02010);textfile_3.write(text_021);

            if VerboseOnScreen: 
                print text_02010,text_021   
        """
        #  <---- OLD 

        #------------------------------------------

        # If the maximum number of events in a given box has
        # not reached, then process it in the rest of this cell
        # script, otherwise discard it and move on to the next star.
        # Also, process the star if it has a change in magnification larger
        # than 1.1 (i.e., 10%).
        if (num_events_int <= num_events_lim_int and
            flag_dim_AA != "x"):

            #------------------------------------------

            # From the main magnification table, upload the information
            # of the magnification of an individual LC.
            # It is a very large file so I use pandas to read the datatable
            # by portions.
                        
            if KindLensEvent == 'PointSource':
                lens_data = pd.read_table(DirLensFiles+MagnificationData,
                              skiprows=initial_row, nrows=number_rows, sep='\s+')
            elif KindLensEvent == 'Binary':
                lens_data = pd.read_table(DirLensFiles+MagnificationData,
                              skiprows=initial_row, nrows=number_rows, sep='\s+')

            if debug:
                text_030 = "Initial_row = %s, number_rows = %s.\n"%(
                    initial_row, number_rows)
                text_033 = "First row of the light curve: %s \n"%(
                    lens_data.values[0])
                textfile_3.write(text_030);textfile_3.write(text_033);
                if VerboseOnScreen: print text_030,text_033

            ##########################################################

            # I use this loop just to count the number of rows and to
            # discard the magnified stars with magnitudes larger than
            # 16 mags for more than a given porcentage of time.

            # Loop over the magnification rows for a given LC in the
            # lensing file, in u-band.

            # Reset the counter for the number of rows for this event
            # based on the change in magnitude in u band:
            countNROW = 0;
            count_AboveMaxMag = 0;
            count_belowMaxDim = 0;

            # kappa of the blending equations.
            # old. kappa_int = kappa_fun(magStar_u_s, magStar_u_2)

            for ii in range(1,len(lens_data)):

                Magnific = lens_data.values[ii][2]
                # old. mag_u = Magnification2mag(Magnific, magStar_u)

                # magnified apparent magnitude from the 2 blended stars.
                # old. mag_u = appmag_magnify_fun(magStar_u_s, Magnific, kappa_int)
                mag_u = appmag_magnify_fun(magStar_u1, Magnific, kappa_u_int)

                # Compute the difference between the previous apparent
                # magnitude and the current value:
                if ii > 1:
                    Magnific_prev = lens_data.values[int(ii-1)][2]
                    # old. mag_u_prev = Magnification2mag(Magnific_prev, 
                    #                   magStar_u)
                    mag_u_prev = appmag_magnify_fun(magStar_u1, 
                                            Magnific_prev, kappa_u_int)
                    diffmag = abs(round(mag_u,round_appmag) -
                                  round(mag_u_prev,round_appmag))
                else: diffmag = 1

                if diffmag >= diffmag_fix:
                    countNROW += 1
                    if mag_u < maxMag:
                        count_AboveMaxMag += 1
                    if mag_u > max_dim_mag:
                        count_belowMaxDim += 1

            # Determine the percentage of MJD with mag larger than
            # the magnitude threshold:
            porcentage_int1 = (float(count_AboveMaxMag)*100.)/(float(countNROW+2))

            # Determine the percentage of MJD with mag dimmer than
            # the magnitude threshold:
            porcentage_dim_int1 = (float(count_belowMaxDim)*100.)/(float(countNROW+2))

            if debug:
                if VerboseOnScreen:
                    print "countNROW = %s | porcentage_int1 = %.2f."%(
                    countNROW, porcentage_int1)
                if countNROW < 1:
                    if VerboseOnScreen:
                        print "Fail because the change in magnitude along the \
entire light curve is \nless than %s mag and/or because countNROW < 10 \
obs in the LC that passed this cut."%diffmag_fix
                if porcentage_int1 > Porcentage_maxTime:
                    if VerboseOnScreen:
                        print "Fail because %.2f percent > %.2f percent limit."%(
                        porcentage_int1, Porcentage_maxTime)

            #########################################

            # Consider the LCs that have less than the maximum
            # porcentage of time of a LC with magnitude larger
            # than the threshold magnitude only.
            # Also, consider only LCs with at least a given
            # number of observations.

            if (porcentage_int1 < Porcentage_maxTime and
                porcentage_dim_int1 < Percentage_obs_MaxDim and
                countNROW > MinNumGoodObs):

                count_succesful_stars = count_succesful_stars + 1

                textfile_1.write(" \n")
                textfile_1.write(text_line)
                textfile_1.write(" \n")
                textfile_1.write("START_EVENT: %s \n"%count_succesful_stars)

                # Write the metadata info for this star.
                textfile_1.write("NROW: %s  RA: %.5f   DEC: %.5f. \n"%(
                    (countNROW+2), ra, dec))

                # Write a text file with the (ra,dec) of this event.
                textfile_2.write("%8.4f  %8.4f \n"%(ra,dec))

                # Determine the "WEIGHT"
                weight_density2 = (float(num_events_lim_fix)/MaxNumLensEvents)**2.
                textfile_1.write("WEIGHT: %.6f \n"%weight_density2)

                # Useful to print and keep track of the LC number in
                # Rosanne metadata
                Rosanne_LC_num = lens_metadata[index_lensFile][0]

                if debug:
                    text_040 = "-WRITING-: ROSANNE LC NUMBER: %s.\n"%(Rosanne_LC_num)
                    text_043 = "LC num = %s | Num of rows = %s \
| Num obs to write = %s.\n"%(Rosanne_LC_num,
                    number_rows, (countNROW+2))
                    text_047 = "CC: %7s = jj  %7s = index_starsFile  \
%7s = index_lensFile.\n"%(jj, index_starsFile, index_lensFile)

                    textfile_3.write(text_040);textfile_3.write(text_043);
                    textfile_3.write(text_047);

                    if VerboseOnScreen:
                        print text_040,text_043

                #-----------------------------------------

                # Write the parameters of the specific event:

                if KindLensEvent == 'PointSource':
                    textfile_1.write("PARVAL: %.4f  %.4f \n"%(
                        lens_metadata[index_lensFile][2],
                        lens_metadata[index_lensFile][3]) )

                elif KindLensEvent == 'Binary':
                    textfile_1.write("PARVAL: %.4f  %.2f  %.2f  \
%.2f  %.4f  %.4f  %.4f  %.4f  %.4f  %.4f  %.4f  %.4f  %.4f  \
%.4f  %.4f  %.4f \n"%(
                        lens_metadata[index_lensFile][3],
                        lens_metadata[index_lensFile][4], 
                        lens_metadata[index_lensFile][5],
                        lens_metadata[index_lensFile][6], 
                        lens_metadata[index_lensFile][7],
                        lens_metadata[index_lensFile][8], 
                        lens_metadata[index_lensFile][9],
                        lens_metadata[index_lensFile][10],  
                        lens_metadata[index_lensFile][11],
                        lens_metadata[index_lensFile][12], 
                        lens_metadata[index_lensFile][13],
                        lens_metadata[index_lensFile][14], 
                        lens_metadata[index_lensFile][15],
                        lens_metadata[index_lensFile][16], 
                        lens_metadata[index_lensFile][17],
                        lens_metadata[index_lensFile][18]        
                    ) )

                if debug:
                    if VerboseOnScreen:
                        print "par1 = %.4f | par2 = %.4f."%(
                        lens_metadata[index_lensFile][2],
                        lens_metadata[index_lensFile][3])

                #------------------------------------------

                # Apparent magnitude star in different bands.
                # Source star
                magStar_u1 = starsdata['f2'][index_starsFile]
                magStar_g1 = starsdata['f3'][index_starsFile]
                magStar_r1 = starsdata['f4'][index_starsFile]
                magStar_i1 = starsdata['f5'][index_starsFile]
                magStar_z1 = starsdata['f6'][index_starsFile]
                magStar_y1 = starsdata['f7'][index_starsFile]

                # 2nd star = the next in the LSST catalog.
                magStar_u2 = starsdata['f2'][index_starsFile+1]
                magStar_g2 = starsdata['f3'][index_starsFile+1]
                magStar_r2 = starsdata['f4'][index_starsFile+1]
                magStar_i2 = starsdata['f5'][index_starsFile+1]
                magStar_z2 = starsdata['f6'][index_starsFile+1]
                magStar_y2 = starsdata['f7'][index_starsFile+1]

                # Compute kappa for the blended stars:
                kappa_u_int = kappa_fun(magStar_u1, magStar_u2)
                kappa_g_int = kappa_fun(magStar_g1, magStar_g2)
                kappa_r_int = kappa_fun(magStar_r1, magStar_r2)
                kappa_i_int = kappa_fun(magStar_i1, magStar_i2)
                kappa_z_int = kappa_fun(magStar_z1, magStar_z2)
                kappa_y_int = kappa_fun(magStar_y1, magStar_y2)

                # Compute the base app mag from the blended stars:
                magStar_u = appmag_base_fun(magStar_u1, kappa_u_int)
                magStar_g = appmag_base_fun(magStar_g1, kappa_g_int)
                magStar_r = appmag_base_fun(magStar_r1, kappa_r_int)
                magStar_i = appmag_base_fun(magStar_i1, kappa_i_int)
                magStar_z = appmag_base_fun(magStar_z1, kappa_z_int)
                magStar_y = appmag_base_fun(magStar_y1, kappa_y_int)

                #------------------------------------------

                # old_1: time_first = lens_data.values[1][1] - 1.0 # initial time
                time_first = lens_data.values[0][1] # initial time

                # Magnitudes of the template
                textfile_1.write("T: %11.4f   %.3f  %.3f  %.3f  %.3f  %.3f  \
%.3f \n"%(time_first, magStar_u, magStar_g, magStar_r, magStar_i,
                    magStar_z, magStar_y))

                if debug: 
                    if VerboseOnScreen:
                        print "Initial time template = %.4f."%time_first

                # OLD:
                # Write the first event: it is equal in mags to template
                # textfile_1.write("S: %9.4f  %.3f  %.3f  %.3f  %.3f  %.3f  \
                # %.3f \n"%(
                #     time_first, magStar_u, magStar_g, magStar_r, magStar_i,
                #     magStar_z, magStar_y))

                #---------------------------------------------------
                # Loop over the magnification file for a given star

                if plot_LightCurves:
                    time_list = []
                    mag_u_list = [];
                    mag_u_list_NoBlend = []; # With NO blending

                    time_list  += [time_first]
                    mag_u_list += [magStar_u]
                    mag_u_list_NoBlend += [magStar_u1]

                if debug:
                    if VerboseOnScreen:
                        print "len(lens_data) = %s."%len(lens_data)
                        print "First row of values in 'lens_data' array:"
                        print "%i | %.4f | %.3f."%(
                            lens_data.values[0][0],lens_data.values[0][1],
                            lens_data.values[0][2])

                # Reset dummy counter to track the values of i3 in the
                # loop below.
                count_int3 = 0;
                # Count the number of observations that pass
                # the 'diffmag >= diffmag_fix' cutoff
                countNROW_2 = 0;

                # Loop over the magnification data.
                for i3 in range(1,len(lens_data)):

                    if debug and count_int3 < 1:
                        if VerboseOnScreen:
                            print "First rows of values read in this loop:"
                            print "%i | %.4f | %.3f | loop index i3 = %s."%(
                                lens_data.values[i3][0],lens_data.values[i3][1],
                                lens_data.values[i3][2], i3)

                    time_int = lens_data.values[i3][1]
                    Magnific = lens_data.values[i3][2]

                    # Compute the magnified app mag from the blended stars:
                    mag_u = appmag_magnify_fun(magStar_u1, Magnific, kappa_u_int)
                    mag_g = appmag_magnify_fun(magStar_g1, Magnific, kappa_g_int)
                    mag_r = appmag_magnify_fun(magStar_r1, Magnific, kappa_r_int)
                    mag_i = appmag_magnify_fun(magStar_i1, Magnific, kappa_i_int)
                    mag_z = appmag_magnify_fun(magStar_z1, Magnific, kappa_z_int)
                    mag_y = appmag_magnify_fun(magStar_y1, Magnific, kappa_y_int)

                    # No blending:
                    mag_u_NoB = Magnification2mag(Magnific, magStar_u1)
                    mag_g_NoB = Magnification2mag(Magnific, magStar_g1)
                    mag_r_NoB = Magnification2mag(Magnific, magStar_r1)
                    mag_i_NoB = Magnification2mag(Magnific, magStar_i1)
                    mag_z_NoB = Magnification2mag(Magnific, magStar_z1)
                    mag_y_NoB = Magnification2mag(Magnific, magStar_y1)

                    # Compute the difference between the previous magnitude
                    # and the current value, and print only the times
                    # when the difference is larger than "diffmag_fix"
                    if i3 > 1:
                        Magnific_prev = lens_data.values[int(i3-1)][2]
                        mag_u_prev = appmag_magnify_fun(magStar_u1, 
                                        Magnific_prev, kappa_u_int)
                        diffmag = abs(round(mag_u,round_appmag) -
                                      round(mag_u_prev,round_appmag))
                    else: diffmag = 1

                    if diffmag >= diffmag_fix:

                        # Write the rows:
                        text_05 = 'S: %11.4f   %.3f  %.3f  %.3f  %.3f  %.3f  %.3f \n'%(
                            time_int, mag_u, mag_g, mag_r, mag_i, mag_z, mag_y)
                        textfile_1.write(text_05)

                        # Count the number of observations that pass
                        # the 'diffmag >= diffmag_fix' cutoff
                        countNROW_2 += 1

                        if debug and countNROW_2==1:
                            if VerboseOnScreen:
                                print "Num obs = %i | Second time = %.4f. | \
Magnific = %.3f."%(lens_data.values[i3][0], time_int, Magnific)

                        if plot_LightCurves:
                            time_list  += [time_int]
                            mag_u_list += [mag_u]
                            mag_u_list_NoBlend += [mag_u_NoB]

                        # A hack to define the last MJD. I will use this to
                        # write down the last event where the MJD will the the
                        # last used time + 1 day, and the magnitudes will
                        # be the same than the template.
                        time_int2 = time_int

                    # Dummy counter to track the very first value
                    # in this loop.
                    count_int3 +=1

                # Write the LAST date of the event. 
                # Define the last date in Rosanne's LC file:
                lastDate_int = lens_data.values[-1][1]
                # Define the date before the last one.
                lastDate_prev = lens_data.values[-2][1]
                # Define the difference between the last 2 dates:
                lastDate_diff = lastDate_int - lastDate_prev
                # Define 5% of the diff between the last 2 dates: 
                increase_int1 = (lastDate_diff)*(percent_increaseLastDate/100.0)
                # The last date I'll write in the file = the difference in time
                # between the last 2 dates + a 10% increase. This to ensure
                # smoothness in the last part of the light curve.
                if KindLensEvent == 'PointSource':
                    lastDate_write = lastDate_int + lastDate_diff + increase_int1
                elif KindLensEvent == 'Binary':
                    lastDate_write = lastDate_int + lastDate_diff + increase_int1
                
                textfile_1.write("S: %11.4f   %.3f  %.3f  %.3f  %.3f  %.3f  \
%.3f \n"%(lastDate_write, magStar_u, magStar_g, magStar_r, magStar_i,
                    magStar_z, magStar_y))

                textfile_1.write("END_EVENT: %s \n"%count_succesful_stars)

                if plot_LightCurves:
                    time_list  += [lastDate_write]
                    mag_u_list += [magStar_u]
                    mag_u_list_NoBlend += [magStar_u1]

                # if debug:
                #     print "j = %s: countNROW = %s, countNROW_2 = %s."%(
                #         j, countNROW,countNROW_2)

                # Given that this star passed all the cuts then add one
                # count to the box where the star is located:
                count_expectedEvents_np[box_row_index][5] += 1


                if num_events_int == num_events_lim_int:
                    # Count the number of boxes has been completely filled so far.
                    count_boxesFull +=1 
                    append_4 = '# Box full. %s boxes full so far in total.'%count_boxesFull
                else: append_4 = ''

                text_083 = "%10s = j  %10s = PLASTICC event  %10s LSST star  \
%10s = Rosanne LC number  %10s = index_lensFile | written | %s \n"%(
                    jj, count_succesful_stars, index_starsFile, Rosanne_LC_num, index_lensFile,
                    append_4)
                
                # ---- MAIN REPORT ---->>
                text_084 = "%10s %10s %10s %10i %10s    %10i  %10i  %12i  %4i   %8.4f %8.4f  \
%11s   %.2f  %7i      %12.5f     %12.5f   %s # %s\n"%(
                    jj, count_succesful_stars, index_starsFile, Rosanne_LC_num, 
                    index_lensFile,num_events_int, num_events_lim_int,
                    num_events_lim_fix, box_row_index,
                    ra, dec, initial_row, lens_data.values[0][0], number_rows,
                    AA_at_peak_int, AA_total_max_r_int, flag_dim_AA,
                    append_4)
                # <<---- MAIN REPORT ----

                if debug:  
                    text_07980 = "DD: %7s = jj  %7s = index_starsFile  \
%7s = index_lensFile.\n"%(jj, index_starsFile, index_lensFile) 
                    text_080 = "Box number (= row) %s: added one count.\n"%box_row_index
                    text_085 = "SUCCESS %s! \n"%count_succesful_stars

                    textfile_3.write(text_080);
                    textfile_3.write(text_083); textfile_3.write(text_084);
                    textfile_3.write(text_07980);
                    textfile_3.write(text_085);

                    if VerboseOnScreen:
                        print text_080,text_083;text_07980;
                        print text_084;
                        print text_085;

                else: # ---- MAIN REPORT ----:
                    textfile_3.write(text_084);

                #------------- PLOT THE LCs -------------------
                
                # Plot every n-th sucessful events. Define a useful parameter
                # for that:
                ratio_int1 = round((float(count_succesful_stars)/float(PlotEvery_n)),3)
                
                if (plot_LightCurves and 
                    ratio_int1==int(ratio_int1)):
                    
                    # print "%s = ratio_int1: %s"%(ratio_int1, 
                    #          ratio_int1==int(ratio_int1))

                    # Creating the plot
                    plt.figure()

                    # Plot with points and lines (recommended)
                    plt.plot(time_list, mag_u_list, lw=0.5, marker=".", ms=2,
                            color = 'red', label='With blending')

                    # just the line:
                    # plt.plot(time_list, mag_u_list, lw=2, ls = "-", color = 'blue')
                    # plt.plot(time_list, mag_g_list, lw=2, ls = "-", color = 'green')

                    plt.grid(ls='--', alpha=0.3)
                    plt.xlim(min(time_list)-5, max(time_list)+5)
                    plt.ylim(max(mag_u_list)+0.05, min(mag_u_list)-0.1)

                    plt.xlabel('time (days)')
                    plt.ylabel('apparent magnitude')
                    plt.legend(loc='upper left')

                    plt.title('u-band lensed and blended %s LSST star (%s Rosanne LC)'%(
                        index_starsFile, Rosanne_LC_num))

                    plt.savefig(DirSaveOutput+'%s_plots/plot_%s_LSST_%s_Rosanne_%s.png'%(
                        count_run,count_succesful_stars, index_starsFile, Rosanne_LC_num), dpi=120)
                    
                    # Overplot the magnificated 1 star with NO blending
                    if plot_WithNoBlending:
                        plt.plot(time_list, mag_u_list_NoBlend, lw=0.5, marker=".", ms=2,
                            color = 'blue', label='No blending')
                    plt.ylim(max(mag_u_list)+0.6, min(mag_u_list)-0.1)
                    
                    plt.savefig(DirSaveOutput+'%s_plots/plot_%s_LSST_%s_Rosanne_%s_over.png'%(
                        count_run,count_succesful_stars, index_starsFile, Rosanne_LC_num), dpi=120)
                        
                    plt.close()
                    
                #------------------------
                    
                if debug:
                    text_087010 = "# Hola after plot section (even with no plotting).\n"
                    textfile_3.write(text_087010);

            #--------------------------------------
            # Update the value of "initial_row"
            initial_row = initial_row + number_rows
            # Flag that "initial_row" has been updated here.
            initial_row_updateFlag = 1
            
            if debug:
                text_087020 = "# Hola after update 'initial_row' to the value %s.\n"%(
                    initial_row)
                textfile_3.write(text_087020);

            # Update counter to be use as index read the lines in the
            # lensing metadata array.
            index_lensFile += 1

            if debug:
                text_087030 = "# Hola after update 'index_lensFile' to the value %s.\n"%(
                    index_lensFile)
                textfile_3.write(text_087030);
                
                if VerboseOnScreen:
                    print text_087030

    #====================================================
    # Update the index used to read each row in the LSST
    # star catalog.
    index_starsFile += 1
    
    if debug:
        text_087040 = "# Hola after update 'index_starsFile' to the value %s.\n"%(
            index_starsFile)
        textfile_3.write(text_087040);

    # Update the index used to read each row in Rosanne
    # metadata file if the magnification is smaller than a cutoff
    if (AA_at_peak_int < Min_TotalMagnific and
        initial_row_updateFlag == 0):        
        # Update the value of "initial_row":
        initial_row = initial_row + number_rows
        index_lensFile += 1
    
        if debug:
            text_087045 = "# Hola after update 'index_lensFile' to the value %s.\n"%(
                index_lensFile)
            textfile_3.write(text_087045);
    
    #--------------------------------------------------
    
    # Break when the number of desired number of events to write down 
    # in the PLASTICC file has been reach. Or when all the boxes are full of
    # events.
    if count_succesful_stars == TargetNumberEvents:
        text_0910 = "\n\n# Target number of events (%i) reached :) ! \n"%count_succesful_stars
        textfile_3.write(text_0910)
        print text_0910
        break
        
    if count_boxesFull == len(expectedEventsLSST_np):
        text_0920 = "\n\n# All %i boxes have been filled of events :) ! \n"%count_boxesFull
        textfile_3.write(text_0920)
        print text_0920
        break
    
##################################################

# Write timing

text_16 = "\n#     Time Summary \n"
text_17 = "# Starting time of computing: %s \n"%time_start

textfile_3.write("# \n")
now_end = datetime.datetime.now()
time_end = now_end.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
text_18 = "# Ending time of computing: %s \n"%time_end

# Compute the time the  computations took:
time_compute = now_end - now_start
text_19 = "# Total time of computing:  %s \n"%time_compute

textfile_3.write(text_line)
textfile_3.write(text_16);textfile_3.write(text_17);
textfile_3.write(text_18);textfile_3.write(text_19);
# textfile_3.write()

#---------------------------

textfile_3.write(text_line);
textfile_3.write(text_008010);textfile_3.write(text_008014);
textfile_3.write(text_00802);textfile_3.write(text_00804);
textfile_3.write(text_0080405);
textfile_3.write(text_0080410);
textfile_3.write(text_0080420);textfile_3.write(text_0080430);
textfile_3.write(text_00806);textfile_3.write(text_00808);
textfile_3.write(text_00810);textfile_3.write(text_00812);
textfile_3.write(text_00814);textfile_3.write(text_00816);
textfile_3.write(text_00818);textfile_3.write(text_00819);
textfile_3.write(text_00820);

#---------------------------
text_20 = "\njj = %s: last iteration in the loop. \n"%jj
text_100 = "Last light curve and star used:\n"
text_22 = "\n# %s total succesful stars written in the PLASTICC file. \n"%count_succesful_stars

textfile_3.write(text_20);
textfile_3.write(text_100);textfile_3.write(text_083);
textfile_3.write(text_22)
#---------------------------

textfile_1.close(); textfile_2.close(); textfile_3.close();

print text_line, text_16,text_17, text_18, text_19
print text_100, text_083, text_22

print "# Loop finished :) "

# Number of useful rows written in the text file for a given star,
# it is, when there is a change in magnitude at least > 0.001 mag.
#------------------------------

#     Time Summary 
# Starting time of computing: 2018-08-01 (yyyy-mm-dd); 23:36 hrs. 
# Ending time of computing: 2018-08-01 (yyyy-mm-dd); 23:36 hrs. 
# Total time of computing:  0:00:33.391188 

Last light curve and star used:
        95 = j          24 = PLASTICC event          95 LSST star        45.0 = Rosanne LC number          44 = index_lensFile | written |  

# 24 total succesful stars written in the PLASTICC file. 

# Loop finished :) 


In [38]:
# Uncomment and run this cell in case there is an error during the run
# of the above main loop cell.

"""
# Write timing

text_16 = "\n#     Time Summary \n"
text_17 = "# Starting time of computing: %s \n"%time_start

textfile_3.write("# \n")
now_end = datetime.datetime.now()
time_end = now_end.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
text_18 = "# Ending time of computing: %s \n"%time_end

# Compute the time the  computations took:
time_compute = now_end - now_start
text_19 = "# Total time of computing:  %s \n"%time_compute

textfile_3.write(text_line)
textfile_3.write(text_16);textfile_3.write(text_17);
textfile_3.write(text_18);textfile_3.write(text_19);
# textfile_3.write()

#---------------------------

textfile_3.write(text_line)
textfile_3.write(text_008010);textfile_3.write(text_008014);
textfile_3.write(text_00802);textfile_3.write(text_00804);
textfile_3.write(text_0080410);
textfile_3.write(text_0080420);textfile_3.write(text_0080430);
textfile_3.write(text_00806);textfile_3.write(text_00808);
textfile_3.write(text_00810);textfile_3.write(text_00812);
textfile_3.write(text_00814);textfile_3.write(text_00816);
textfile_3.write(text_00818);textfile_3.write(text_00819);
textfile_3.write(text_00820);

#---------------------------
text_20 = "\n jj = %s: last iteration in the loop. \n"%jj
text_100 = "Last light curve and star used:\n"
text_22 = "\n# %s total succesful stars written in the PLASTICC file. \n"%countEvent

textfile_3.write(text_20);
textfile_3.write(text_100);textfile_3.write(text_083);
textfile_3.write(text_22)
#---------------------------

print text_line, text_16,text_17, text_18, text_19

"""
0

0

In [39]:
textfile_1.close();textfile_1.close();textfile_1.close();
textfile_1.close();textfile_1.close();textfile_1.close();

textfile_2.close();textfile_2.close();textfile_2.close();
textfile_2.close();textfile_2.close();textfile_2.close();

textfile_3.close();textfile_3.close();textfile_3.close();
textfile_3.close();textfile_3.close();textfile_3.close();

plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();

print text_100, text_083, text_22

#---------------------------
# Count the number of times I run this section. The number
# is used to label the output files while debugging.
count_run += 1 


Last light curve and star used:
        95 = j          24 = PLASTICC event          95 LSST star        45.0 = Rosanne LC number          44 = index_lensFile | written |  

# 24 total succesful stars written in the PLASTICC file. 



## Plot (ra, dec)

In [40]:
# Upload the data

ra_dec_file = '%s_LSST_lensingEvents_RA_DEC_.dat'%(count_run-1)
ra_dec_np = np.genfromtxt(DirSaveOutput+ra_dec_file)
print "len(ra_dec_np) = %s"%len(ra_dec_np)

len(ra_dec_np) = 24


In [41]:
#    PLOTTING

# Overplot a grid of the size = Size_subdivision?
plotgridsize = True

## If 'plotgridsize = False' then what subdivision size to use?
Size_subdivision = 10 # degrees

#-------------------------------
# Color points

# Mycolor_points = 'red'
Mycolor_points = 'black'
# Mycolor_points = 'blue'

#-------------------------------

fig = plt.figure() 

plt.plot(ra_dec_np[:,0], ra_dec_np[:,1], 
        Mycolor_points, marker='.', ms=1, ls='None', alpha=0.4)

if plotgridsize:
    plt.xticks(np.arange(0, 370, Size_subdivision),rotation=90)
    plt.yticks(np.arange(-90, 50, Size_subdivision))
    plt.tick_params(labelsize=5)
    plt.grid(alpha=0.5, ls='-')
else: plt.grid() 
     
plt.gca().invert_xaxis() # Invert x-axis
plt.xlabel("RA")
plt.ylabel("DEC")
plt.title("LSST events (%s)"%len(ra_dec_np))


if plotgridsize:
    plt.savefig(DirSaveOutput+'%s_Plot_gridsize_%s_.png'%(
        ra_dec_file[:-5],Size_subdivision), dpi=150)
else:
    plt.savefig(DirSaveOutput+'%s_Plot_.png'%(ra_dec_file[:-5]), dpi=100)  

# old. plt.savefig(DirSaveOutput+'%s_Plot_.png'%(ra_dec_file[:-5]), dpi=120)

plt.close()

In [42]:
plt.close();plt.close();plt.close();

# Find out duplicated dates in the output PLASTICC file.

In [43]:
# count_run = 2

In [48]:
# Open to read the main LSST star catalog

# TEMPORAL
"""
FileToCheck = '1_Lensing_lc_PLASTICC_format_.dat'
ReportFile = '1_Report_duplicate_.log'
DirFile = '/Users/arturo/Dropbox/Research/Articulos/14_Lensing/MockData/Production_PointSource/'
DirSaveOutput = DirFile
"""

# ORIGINAL
 
FileToCheck = '%s_Lensing_lc_PLASTICC_format_.dat'%(count_run-1)
ReportFile = '%s_Report_duplicates_.log'%(count_run-1)
DirFile = DirSaveOutput


LSST_events_data = open(DirSaveOutput+FileToCheck, 'r')
textfile_1 = open(DirSaveOutput+ReportFile, 'w')

text_line = '#'+'-'*30 + '\n'
text_010 = "#   Report of duplicated dates \n"
text_020 = "# File: '%s' \n"%FileToCheck
text_030 = "# %s \n"%DirFile

textfile_1.write(text_010);textfile_1.write(text_020)
textfile_1.write(text_030);textfile_1.write(text_line)
print text_010, text_020, text_030, text_line

#-----------------------

count_repeated = 0
count_line = 0
date_int1 = "        "

for line in LSST_events_data: 
    
    count_line += 1
    
    if line[:2] == "S:":
        date_int2 = line[4:14]
        if date_int2 == date_int1:
            #print "# Line %s duplicated: %s"%(count_line,date_int2)
            text_050 = "# Line %8i duplicated: %s"%(count_line,line)
            textfile_1.write(text_050)
            count_repeated += 1
            
        # Update the value.
        date_int1 = date_int2
        
LSST_events_data.close();

text_100 = "# %s repeated dates in -total- in this file. \n"%count_repeated
text_110 = "# %s total rows in this file. \n"%count_line

textfile_1.write(text_line);
textfile_1.write(text_100);textfile_1.write(text_110);
print text_100,text_110

textfile_1.close()

#   Report of duplicated dates 
# File: '2_Lensing_lc_PLASTICC_format_.dat' 
# /Users/arturo/Dropbox/Research/Articulos/14_Lensing/MockData/Production_PointSource/ 
#------------------------------

# 0 repeated dates in -total- in this file. 
# 9107 total rows in this file. 



In [49]:
LSST_events_data.close();LSST_events_data.close();
LSST_events_data.close();LSST_events_data.close();

textfile_1.close(); textfile_1.close(); textfile_1.close();
textfile_1.close(); textfile_1.close(); textfile_1.close();

In [50]:
# File: '3_Lensing_lc_PLASTICC_format_.dat'
# /Users/arturo/Dropbox/Research/Articulos/14_PLASTICC/MockData/Production_Binary/ 

# Line 879429 duplicated: S:     70.4167   23.941  20.756  18.525  17.257  16.322  15.767 

# 1 repeated dates in this file. 

In [51]:
print "# ALL DONE."

# ALL DONE.


-------

# Check if there are lines with issues in Rosanne magnification input file

In [ ]:
import numpy as np
# Function to identify if a string is an integer number or a letter.
# This will be used in the dictionary construction to properly read some SN names.

def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

# Tests
print is_number('5'), is_number('e'), is_number(''), is_number(' '),
# True False False False

In [ ]:
# Open to read the main LSST star catalog

#----- Point source events--------

Rosanne_file = 'tau.dist_lcs_n'
DirRosanne = '/Users/arturo/Documents/Research/LSST/Rosanne/2018_07_24_Point/'
# Column of the last digit in the first column of data in the file.
ColumnLastDigit = 14
# Last row in the file. It was determined from "$ wc -l File.dat"
LastRow = 29826782

#----- Binary events --------
"""
# Rosanne_file = 'bin_lc_results_10_50_n_last_3000.dat'
Rosanne_file = 'tau.dist_lcs_new'
DirRosanne = '/Users/arturo/Documents/Research/LSST/Rosanne/2018_07_25_Point/'
# Column of the last digit in the first column of data in the file.
ColumnLastDigit = 8
# Last row in the file. It was determined from "$ wc -l File.dat"
LastRow = 29826782
"""

#--------------------------------

text_line = '#'+'-'*30 + '\n'
print text_line, "# File: '%s'"%Rosanne_file
print "# %s \n"%DirRosanne

Rosanne_data = open(DirRosanne+Rosanne_file, 'r')

count_row = 0
count_bad = 0
row_bad_list = []

for line in Rosanne_data: 
    
    count_row += 1
    # # print line[ColumnLastDigit]
     
    try:
        if not is_number(line[ColumnLastDigit]):
            print "Row %s: %s"%(count_row, line[:40])
            count_bad += 1
            row_bad_list += [count_row]
    except:
        print "Row %s: Has an issue I even could not print out."%(count_row)
        count_bad += 1
    
    # If I reach the end-1 of the file, then stop the loop:
    if count_row == LastRow: break
    
Rosanne_data.close();

print "# %s total bad rows in this file."%count_bad
print "# %s total rows in this file."%count_row

In [ ]:
Rosanne_data.close();Rosanne_data.close();
Rosanne_data.close();Rosanne_data.close();

In [ ]:
"""
#------------------------------
# File: 'tau.dist_lcs_n'
# /Users/arturo/Documents/Research/LSST/Rosanne/2018_07_24_Point/ 

# 0 total bad rows in this file.
# 19218155 total rows in this file. 

""" 
0

# Determine what is the number of lines that the Rosanne's magnification file should have based on the number of light curves and their corresponding number of entries for each LC described in the metadata file.

In [ ]:
import numpy as np

Rosanne_metada_file = 'bin_lc_information_10_50_n.dms'
DirRosanne = '/Users/arturo/Documents/Research/LSST/Rosanne/2018_07_24_Binary/'

Rosanne_metadata_np = np.genfromtxt(DirRosanne+Rosanne_metada_file)

SumNumRowCol = sum(Rosanne_metadata_np[:,1])
print "# %s rows should have the magnification file."%SumNumRowCol

# 26609211.0 rows should have the magnification file.

In [ ]:
26609211
26645299

In [ ]:
Rosanne_metadata_np[:3,1]

------

# Write to a text file with the -last- date of the magnification for each LC in Rosanne's file.

In [ ]:
# Open to read the main LSST star catalog

#----- Point source events--------


# Rosanne_file = 'tau.dist_lcs_new_head_10000.dat'
# output_file = '%s_LastDates_.dat'%Rosanne_file[:-4]

Rosanne_file = 'tau.dist_lcs_new'
output_file = '%s_LastDates_.dat'%Rosanne_file

DirRosanne = '/Users/arturo/Documents/Research/LSST/Rosanne/2018_07_25_Point/'
# Column of the first and last digit in the column of data.
ColumnFirstDigit = 13
ColumnLastDigit = 15
BreakLoopInThisLCnumber = 2000


#----- Binary events --------

"""
Rosanne_file = 'bin_lc_results_10_50_n.dms'
output_file = '%s_LastDates.dat'%Rosanne_file[:-4]
DirRosanne = '/Users/arturo/Documents/Research/LSST/Rosanne/2018_07_24_Binary/'
# Column of the first and last digit in the column of data.
ColumnFirstDigit = 7
ColumnLastDigit = 9
BreakLoopInThisLCnumber = 2000
"""

#--------------------------------

text_line = '#'+'-'*30 + '\n'
print text_line, "# File: '%s'"%Rosanne_file
print "# %s \n"%DirRosanne

Rosanne_data = open(DirRosanne+Rosanne_file, 'r')
textfile_1 = open(DirRosanne+output_file, 'w')

textfile_1.write("# %s \n"%(Rosanne_file))
textfile_1.write("# %s \n"%(DirRosanne))
textfile_1.write("# In the %s light curve: break the loop. \n"%BreakLoopInThisLCnumber)
textfile_1.write(text_line)
textfile_1.write("#") # Hack to comment the first useless line.

#--------------------------------

count_numLastRows = 0
line_last = '# initial line. \n'

for line in Rosanne_data: 
    
    if line[ColumnFirstDigit:ColumnLastDigit] == ' 1':
        
        text_030 = "%5i   %s"%(count_numLastRows,line_last)
        textfile_1.write(text_030)
        # textfile_1.write(line[ColumnFirstDigit:ColumnLastDigit])
        count_numLastRows += 1
        
    if count_numLastRows == BreakLoopInThisLCnumber: break
        
    line_last = line
    
#--------------------------   
Rosanne_data.close();

text_060 = "# %s total number of 'last lines' of LCs in this file.\n"%count_numLastRows

textfile_1.write(text_line)
textfile_1.write(text_060)

print text_060

In [ ]:
Rosanne_data.close();Rosanne_data.close();Rosanne_data.close();
Rosanne_data.close();Rosanne_data.close();Rosanne_data.close();

textfile_1.close();textfile_1.close();textfile_1.close();
textfile_1.close();textfile_1.close();textfile_1.close();

#### Determine how many LCs have in their last date an magnification larger than 1.001

In [ ]:
magnific_lastDate_np = np.genfromtxt(DirRosanne+output_file) 

print "# %s"%output_file
print "# %s lines in this file."%len(magnific_lastDate_np)

In [ ]:
count_largeMag = 0

for i1 in range(len(magnific_lastDate_np)):
    if magnific_lastDate_np[i1][3] > 1.001: 
        count_largeMag += 1
        
print "# %s LCs have a magnification larger than 1.001 in their last date."%count_largeMag

------

# Quick plots of individual Rosanne's -magnification- LCs

First, create by hand a table using the data from a specific EVENT in the PLASTICC file. Then use the following script to plot it.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
Filename = 'EVENT_1538_point.dat'; rosanne_num = 209

# Filename = 'EVENT_8864_binary.dat'; rosanne_num = 1470
# Filename = "EVENT_4871_point.dat"; rosanne_num = 1554
# Filename = "EVENT_4917_point.dat"; rosanne_num = 1681

DirFile = '/Users/arturo/Dropbox/Research/Articulos/14_Lensing/Issues/Rick/abrutEndOfLCs/Analysis/'

lc_01_np = np.genfromtxt(DirFile+Filename)

print "# %s lines in this file."%len(lc_01_np)

In [ ]:
fig = plt.figure() 

plt.plot(lc_01_np[:,1],lc_01_np[:,2])

plt.grid(ls='--', alpha=0.2)
plt.xlabel("Time (days)")
plt.ylabel("Magnification")
plt.title("%s (%s)"%(Filename[:-4],rosanne_num))

plt.savefig(DirFile+"%s_Magnification_plot.png"%Filename[:-4])
plt.close()

In [ ]:
plt.close();plt.close();plt.close();

# Plot some LCs from the output PLASTICC file directly

## Option 2
## First create a new file with a format easy to read with numpy

In [ ]:
file_name = '1_Lensing_lc_PLASTICC_format_.dat'
newfile_name = 'PLASTICC_LCs_np_.dat'
DirFile = '/Users/arturo/Dropbox/Research/Articulos/14_PLASTICC/MockData/Production_Binary_test/'

read_file = open(DirFile+file_name,'r')
newfile = open(DirFile+newfile_name,'w')

count_events = 0

for line in read_file:
    if line[:11] =='START_EVENT':
        # print line[13:]
        count_events += 1
        newfile.write("%s %s %s %s %s %s %s \n"%(
            line[13:-1],line[13:-1],line[13:-1],
            line[13:-1],line[13:-1],line[13:-1],
            line[13:-1]))
        
        
    if line[:2] =='S:':
        newfile.write("%s"%line[2:])
        
    if line[:9] =='END_EVENT':
        newfile.write("0.0  0.0  0.0  0.0  0.0  0.0  0.0\n")
        newfile.write("\n#-------------------------------\n")

newfile.write("# Number of events:\n")
newfile.write("%s %s %s %s %s %s %s \n"%(count_events,count_events,
                count_events,count_events,count_events,count_events,
                count_events))
read_file.close();
newfile.close();

In [ ]:
read_file.close();read_file.close();read_file.close();
read_file.close();read_file.close();read_file.close();

newfile.close();newfile.close();newfile.close();
newfile.close();newfile.close();newfile.close();

### Now upload and plot the data

In [ ]:
plasticc_np = np.genfromtxt(DirFile+newfile_name)

NumTotalEvents = plasticc_np[-1][1]
print "# %s events this file has."%(NumTotalEvents)
print "# %s lines this file has."%len(plasticc_np)

# 107 lines this file has.

In [ ]:
plasticc_np[:5]